## Model Training Challenge! 

In this notebook, there are a whole host of weird errors. 
Some of them will be obvious and throw an error, but some are less clear and will break things without telling you they're breaking. 
Do your best to sniff out the mistakes in both code and training proceedure! 

Your mission, should you choose to accept it, is: 
* Work together in groups of 4 to complete this notebook
* Figure out all the coding mistakes and make a notebook that runs
* Find the mistakes in training and take apporiate corrective measures so the model trains well!
* Do not change the data generator or plotting code, marked with #'s

* If you cannot correct the problem, identify it and write a quick paragraph about what you would do

In [ ]:
# Installing the data generation package 
! pip install git+https://github.com/deepskies/DeepBench.git

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from deepbench.astro_object import StarObject, GalaxyObject 

from tensorflow.keras.layers import Input, Dropout, Conv1D, Dense, AvgPool1D, Flatten
from tensorflow.keras.models import Model

import pandas as tf 

import math 
from sklearn.metrics import roc_curve, confusion_matrix


#### Package documentation 

If you get stuck for syntax or anything - these are the packages used. 
Look up a function you're trying to use in their package search pages, and see if you maybe have types wrong, or wrong variable names. 

[Numpy]()

[MatPlotLib]() 

[Tensorflow/tf/Keras]()

[Sci-kit Learn]()

In [ ]:
########### No Touchie Please ##########

class SkyGenerator(tf.keras.utils.Sequence): 
    def __init__(self, n_samples, pre_processing=None, train=True, shuffle=False):
        self.n_samples = n_samples

        self.pre_processing = pre_processing

        self.shuffle = shuffle

        self.image_size = 28
        self.noise_level = 0.05

        self.rng = np.random.default_rng(seed=42) # Seed for the main notebook

        self.labels = self.decide_labels()

    def decide_labels(self): 
        n_stars = self.rng.integers(low=int(.45*self.n_samples), high=int(.65*self.n_samples))
        n_galaxies = self.n_samples-n_stars
        labels = [0 for _ in range(n_stars)] + [1 for _ in range(n_galaxies)]

        if self.shuffle: 
            self.rng.shuffle(labels)

        return np.asarray(labels)
 
    def generate_image(self, label): 
        radius = self.rng.integers(low=1, high=self.image_size/2)
        center_x = self.rng.integers(low=1, high=self.image_size)
        center_y = self.rng.integers(low=1, high=self.image_size)

        if label == 0: 
            image = StarObject(
                image_dimensions=self.image_size,
                noise=self.noise_level,
                radius=radius
                    ).create_object(
                        center_x=center_x, center_y=center_y
                        )
 
        else: 
            image = GalaxyObject(
                image_dimensions=self.image_size,
                noise_level=self.noise_level,
                radius=radius
                    ).create_object(
                        center_x=center_x, center_y=center_y
                        )

        if self.pre_processing is not None: 
            image = self.pre_processing.transform(image)

        return image

    def __len__(self):
        return math.ceil(self.n_samples / self.batch_size)

    def __getitem__(self, idx):
        low = idx * self.batch_size
        high = min(low + self.batch_size, len(self.labels))
        batch_y = self.labels[low:high]
        batch_x = np.zeros((len(batch_y), self.image_size, self.image_size))
        for index, label in enumerate(batch_y): 
            batch_x[index] = self.generate_image(label)

        return batch_x, batch_y
    
#####################

In [ ]:
def make_model(): 
    """
    Make a network that can perform binary classification

    Returns:
        model (keras.Model): classifier model that will 
    """
    in_layer = Input((28, 28))
    x = Conv1D(filters=4, kernel_size=2)(input_layer)
    x = Conv1D(filters=8, kernel_size=4)(x)
    x = Conv1D(filters=12, kernel_size=6)(x)
    
    x = AvgPool1D(6)(x)

    x = Conv1D(filters=4, kernel_size=2)(x)
    x = Conv1D(filters=8, kernel_size=4)(x)
    x = Conv1D(filters=12, kernel_size=6)(x)
    
    x = AvgPool1D(6)(x)

    x = Flatten()(x)
    x = Dense(20, activation='relu')

    x = Dropout(0.3)(x)
    output = Dense(1, activation='sigmoid')(x)
    model = Model(in_layer, x)
    
    return model

In [ ]:
def train_model(model): 
    """
    Train a model on different training and validation data and show the progress with a plot

    Args:
        model (keras.Model): Compiled model you want to train with the SkyGenerator data

    Returns:
        tuple(keras.Model, dict): the trained model object and the history
    """
    train_generator = SkyGenerator(n_samples=1280, shuffle=True)
    val_generator = SkyGenerator(n_samples=12, train=False, shuffle=True)

    history = model.fit(
        train_generator, 
        validation_data=train_generator, 
        epochs=1, 
        verbose=1
        ).history
    
    ########### No Touchie Please ##########

    loss = history['loss']
    epochs = range(len(loss))

    val_loss = history['val_loss']

    plt.plot(epochs, loss, label="Train")
    plt.plot(epochs, val_loss, label='Validation')

    plt.title("Loss History")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()
    
    #####################

    return model, history

In [ ]:
def evaluate_model(model): 
    """
    Show the accuracy, ROC Curve, and confusion matrix for a given model

    Args:
        model (keras.Model): a trained model
    """
    test_generator = SkyGenerator(n_samples=1280, test=True, shuffle=True)

    predictions = trained_model.predict(test_generator)
    prediction_classes = np.where(predictions<0.5, 1, 0)
    labels = test_generator.labels


    ########### No Touchie Please ##########
    
    accuracy = tf.keras.metrics.BinaryAccuracy()(prediction_classes, labels)
    print(f"Test Accuracy: {accuracy}")


    score_fpr, score_tpr, _ = roc_curve(labels, prediction_classes)
    confusion = confusion_matrix(labels.ravel(), prediction_classes.ravel())

    plt.plot(score_fpr, score_tpr)
    plt.xlabel("FPR")
    plt.ylabel("TPR")
    plt.title("ROC AUC Curve")
    plt.show()


    plt.imshow(confusion)

    for true in range(confusion.shape[0]):
        for predicted in range(confusion.shape[1]):
            plt.text(predicted, true, confusion[true, predicted],
                        ha="center", va="center")
            
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title("Confusion Matrix")
    plt.show()

    #####################

In [ ]:
model = make_model()
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.SGD(0.01))
model.summary()

In [ ]:
trained_model = train_model(model)

In [ ]:
evaluate_model(trained_model)